In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os
import wget
import pandas as pd
import numpy as np

In [87]:
#importa lista locali FBi
lista_locali = pd.read_csv(r"query_result.csv", sep=",")
links=lista_locali["paginafb"]
type(links)
links_ready=np.array(links)
links_ready
lista_locali.head(10)

,id,paginafb
0,2128788,https://www.facebook.com/GondolierLondon/?page...
1,2128790,https://www.facebook.com/pages/category/Family...
2,2128792,https://www.facebook.com/pages/category/Americ...
3,2128794,https://facebook.com/pages/category/Tex-Mex-Re...
4,2128797,https://www.facebook.com/crackerbarrel.londonky/
5,2128801,https://facebook.com/pages/category/Restaurant...
6,2128802,https://facebook.com/LondonAutoTruckCenter/
7,2128804,https://facebook.com/FazolisLondonKY/
8,2128806,https://www.facebook.com/GoldenCorralLondonKY/
9,2128807,https://facebook.com/compadreslocallondonky/


In [6]:
restaurant_folder =[]
for i in links_ready[:10]:
    i = i.replace("https://www.facebook.com", "")
    i = i.replace("https://facebook.com", "")
    i = i.replace("/pages/category", "")
    i = i.replace("/", "",1)
    #i = i.replace("/")
    i = i.split("/", 1)
    restaurant_name = i[:1]
    restaurant_folder.append(restaurant_name)
restaurant_folder = np.array(restaurant_folder)
restaurant_folder = restaurant_folder.flatten()
for i in restaurant_folder:
    print(i)

GondolierLondon
Family-Style-Restaurant
American-Restaurant
Tex-Mex-Restaurant
crackerbarrel.londonky
Restaurant
LondonAutoTruckCenter
FazolisLondonKY
GoldenCorralLondonKY
compadreslocallondonky


In [174]:
#create folders for output
#SCRAPED MENU
path = os.getcwd()
path = os.path.join(path, "scrapemen")
os.mkdir(path)
path

'/Users/matteozoli/Desktop/SCRAPING/scrapemen'

In [194]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
driver.get("http://www.facebook.com")

time.sleep(1.5)
l=driver.find_element_by_xpath("//*[@title='Allow All Cookies']")
l.click()

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("rimorchiatorediasmatiche@live.it")
password.clear()
password.send_keys("notterosa")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#We are logged in!



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/matteozoli/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if __name__ == '__main__':


In [195]:
start = time.time()
#Scrape menus
anchors = []

df = pd.DataFrame([], columns = ["id", "e-mail"])
for index, row in lista_locali[400:410].iterrows():
    menu_scraped = driver.get(str(row['paginafb'])+"menu")#+"menu/")
    time.sleep(3)
    n_scrolls = 2
    for j in range(1,n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #this will slow the images scrolling, it gets to the bottom of the batch getting loaded
        time.sleep(1)
        #driver.execute_script("window.scrollTo(0, 4000);")
        try:
            anchors = driver.find_elements_by_tag_name('img')                        
            anchors = [a.get_attribute('src') for a in anchors]
        except ValueError:
            print("Oops!  That was no valid.  Try again...")
        #get link for download
        for a in anchors[2:]:
            if "data:image" not in a:
                print(a)
                driver.get(a)
                img = driver.find_elements_by_tag_name("img")
                for k in img:
                    k.get_attribute("src")
    #create path for each loop based on id
        path_2 = os.path.join(path, str(row['id']))
        print(row['id'])
        try:
            os.mkdir(path_2)
        except FileExistsError:
            print("[Errno 17] File exists: '/Users/matteozoli/Desktop/SCRAPING/scrapemen/{}".format(row['id']))
        #download images
        counter = 0
        for image in anchors[2:]:
            save_as = os.path.join(path_2 + "/" + str(counter) + '.jpeg')
                    #any(email_service in user_email for email_service in email_services
            try:                        
                wget.download(image, save_as)
            except ValueError:
                print("Oops!  That was no valid.  Try again...")
            counter += 1
        
        
print(f"Time: {time.time() - start}")



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  app.launch_new_instance()


https://static.xx.fbcdn.net/rsrc.php/v3/yU/r/D2y-jJ2C_hO.png


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


https://static.xx.fbcdn.net/rsrc.php/v3/y5/r/duk32h44Y31.png
https://static.xx.fbcdn.net/rsrc.php/v3/y8/r/he-BkogidIc.png
https://static.xx.fbcdn.net/rsrc.php/v3/yD/r/lVijPkTeN-r.png
https://static.xx.fbcdn.net/rsrc.php/v3/yH/r/kyCAf2jbZvF.png
https://static.xx.fbcdn.net/rsrc.php/v3/yu/r/eXC82ZeepQ7.png
https://static.xx.fbcdn.net/rsrc.php/v3/yY/r/XxEsb0x8INQ.png
https://scontent.fblq5-2.fna.fbcdn.net/v/t1.6435-1/p160x160/91276502_2956916831034144_5804765402827325440_n.jpg?_nc_cat=107&ccb=1-5&_nc_sid=7206a8&_nc_ohc=-KmSHv315mUAX-eldcG&_nc_ht=scontent.fblq5-2.fna&oh=00_AT-mIQDN64R6_Ra5lEKMbBQsRHXoc12MBWH836Xyt_PIcQ&oe=620D9005
https://scontent.fblq5-1.fna.fbcdn.net/v/t15.5256-10/s280x280/272095762_286186453408696_7415873689917860802_n.jpg?_nc_cat=101&ccb=1-5&_nc_sid=ad6a45&_nc_ohc=fm4W8a1rP4QAX8gYOVc&_nc_ht=scontent.fblq5-1.fna&oh=00_AT99XAro0PhoLKv444k7hy4sBsmbQsj2E8YgJWGjSiw1EQ&oe=61EC603A
https://scontent.fblq5-1.fna.fbcdn.net/v/t39.30808-1/cp0/p40x40/222301501_10223774590126131_5109

In [152]:
conter = 0
substrings = ["ata", "xx"]
df = pd.DataFrame([], columns = ["id", "e-mail"])
for index, row in lista_locali[100:200].iterrows():        
    for i in substrings:
        #any(email_service in user_email for email_service in email_services)
        if str(i) not in image:
            wget.download(image, save_as)
            counter += 1  
            print(i)
        else:
            continue
        counter += 1

AttributeError: 'list' object has no attribute 'decode'

In [121]:
image

"data:image/svg+xml,%3csvg xmlns='http://www.w3.org/2000/svg' xmlns:xlink='http://www.w3.org/1999/xlink' viewBox='0 0 16 16'%3e%3cdefs%3e%3clinearGradient id='a' x1='50%25' x2='50%25' y1='0%25' y2='100%25'%3e%3cstop offset='0%25' stop-color='%2318AFFF'/%3e%3cstop offset='100%25' stop-color='%230062DF'/%3e%3c/linearGradient%3e%3cfilter id='c' width='118.8%25' height='118.8%25' x='-9.4%25' y='-9.4%25' filterUnits='objectBoundingBox'%3e%3cfeGaussianBlur in='SourceAlpha' result='shadowBlurInner1' stdDeviation='1'/%3e%3cfeOffset dy='-1' in='shadowBlurInner1' result='shadowOffsetInner1'/%3e%3cfeComposite in='shadowOffsetInner1' in2='SourceAlpha' k2='-1' k3='1' operator='arithmetic' result='shadowInnerInner1'/%3e%3cfeColorMatrix in='shadowInnerInner1' values='0 0 0 0 0 0 0 0 0 0.299356041 0 0 0 0 0.681187726 0 0 0 0.3495684 0'/%3e%3c/filter%3e%3cpath id='b' d='M8 0a8 8 0 00-8 8 8 8 0 1016 0 8 8 0 00-8-8z'/%3e%3c/defs%3e%3cg fill='none'%3e%3cuse fill='url(%23a)' xlink:href='%23b'/%3e%3cuse fil

In [63]:
    path_2 = os.path.join(path, str(row['id']))
    os.mkdir(path_2)
    counter = 0
    for image in anchors[2:]:
        save_as = os.path.join(path_2 + "/" + str(counter) + '.jpeg')
        wget.download(image, save_as)
        counter += 1

100% [..........................................................] 24701 / 24701

In [30]:
                counter = 0
                for image in anchors[2:]:
                    save_as = os.path.join(path + "/" + str(counter) + '.jpeg')
                    wget.download(image, save_as)
                    counter += 1

NameError: name 'path_2' is not defined

In [ ]:
       for i in restaurant_folder:
                path_2 = os.path.join(path, i)
                os.mkdir(path_2)
        #download images and put in right directory
                counter = 0
                for image in anchors[2:]:
                    save_as = os.path.join(path_2 + "/" + str(counter) + '.jpeg')
                    wget.download(image, save_as)
                    counter += 1
    time.sleep(5)

In [126]:
counter = 0
for image in anchors[2:]:
        save_as = os.path.join(path_3 + "/" + str(counter) + '.jpeg')
        wget.download(image, save_as)
        counter += 1

100% [..........................................................] 23997 / 23997

In [153]:
print("Created by Matte Zoli")

Created by Matte Zoli
